# **Store Data to Vector Store (OJK)**

Ini cara untuk storing ke Redis, tapi untuk [Load](#load) Document beda-beda untuk tiap data BI, OJK, dan SIKEPO. Jadi buat sendiri function `extract_all_documents_in_directory` nya

## **Setup**

In [1]:
from dotenv import load_dotenv
load_dotenv()

True

## **Config**

In [2]:
from utils.config import get_config
from utils.models import ModelName, get_model

config = get_config()

## **Define Model**

In [3]:
model_name = ModelName.AZURE_OPENAI
llm_model, embed_model = get_model(model_name=model_name, config=config)

## **Indexing**

In [4]:
documents_dir = './data/documents/'
pickle_path = './data/pickles/'
metadata_path = './data/metadata/files_metadata.csv'

LOAD_PICKLE = True

### **Load**

Untuk SIKEPO dan BI beda cara extract documentsnya, file document_extractor buat sendiri :D.

In [12]:
from utils.documents_extractor.documents_extract_ojk import extract_all_documents_in_directory

if not LOAD_PICKLE:
    documents = extract_all_documents_in_directory(documents_dir, metadata_path, treshold=0.98)

### **Split**

In [5]:
from utils.documents_split import document_splitter
import pickle


if not LOAD_PICKLE:
    all_splits = document_splitter(docs=documents)
    all_splits1 = sorted(all_splits, key=lambda x: (x.metadata['doc_id'], x.metadata.get('page_number', '0')))
    # Open a file and use dump() 
    with open(pickle_path + 'documents1.pkl', 'wb') as file:

        # A new file will be created
        pickle.dump(all_splits1, file) 

# Open the file in binary mode 
with open(pickle_path + 'documents1.pkl', 'rb') as file:
    
    # Call load method to deserialze 
    all_splits = pickle.load(file)

In [6]:
len(all_splits)

113052

### **Storing**

In [10]:
from databases.vector_store import RedisIndexManager

redis = RedisIndexManager(index_name='ojk', embed_model=embed_model, config=config, db_id=0)

redis.delete_all()
redis.store_vector_index(docs=all_splits[0:300], batch_size=200) # Kalau error 'Redis failed to connect: Index does not exist.' ubah isi start_store_idx_indexname.txt menjadi 0
vector_store = redis.load_vector_index()

Start loading from idx: 0
Loaded 1-200 documents
Loaded 201-300 documents


In [16]:
from retriever.retriever_ojk.retriever_ojk import get_retriever_ojk

retriever = get_retriever_ojk(vector_store=vector_store, top_n=7, top_k=20, llm_model=llm_model, embed_model=embed_model, config=config)

In [17]:
retriever.invoke(input='Berikan peraturan yang membahas mengenai pasar modal')

[Document(metadata={'id': 'doc:ojk:187c4e6aa23c41d099e44c8f2056ed71', 'title': 'Dasar Penilaian Investasi Dana Pensiun', 'sector': 'IKNB', 'subsector': 'Dana Pensiun', 'regulation_type': 'Surat Edaran OJK', 'regulation_number': '4/SEOJK.05/2024', 'effective_date': '1 Juli 2024', 'file_url': 'https://www.ojk.go.id/id/regulasi/Documents/Pages/Dasar-Penilaian-Investasi-Dana-Pensiun/SEOJK%204-SEOJK.05-2024%20Dasar%20Penilaian%20Investasi%20Dana%20Pensiun.pdf', 'doc_id': '1', 'page_number': '1', 'relevance_score': 0.2107663}, page_content='sebagaimana dimaksud dalam undang-undang mengenai surat \nutang negara dan surat berharga syariah negara sebagaimana \ndimaksud dalam undang-undang mengenai surat berharga syariah \nnegara. \n4. \nReksa Dana adalah wadah yang dipergunakan untuk menghimpun \ndana dari masyarakat pemodal untuk selanjutnya diinvestasikan \ndalam portofolio efek oleh manajer investasi sebagaimana dimaksud \ndalam undang-undang mengenai pasar modal. \n5. \nMedium Term Notes ya